# Exploration of Random Forest

In [10]:
import os

import pandas as pd
pd.options.mode.chained_assignment = None

import numpy as np
import itertools  

from sklearn.ensemble import RandomForestRegressor

from pprint import pprint

import matplotlib.pyplot as plt
%matplotlib inline 
import time, math

In [11]:
IMC_basedir = os.getcwd().split('code')[0]
DATA_FILE = os.path.join(IMC_basedir, 'data/imputed_dataset_no_censoring_26022018_Amelia1.csv')
DATA_MICE_FILE = os.path.join(IMC_basedir, 'data/imputed_dataset_no_censoring_26022018_MICE')
TRAIN_FILE = os.path.join(IMC_basedir, 'data/amelia_train')
TEST_FILE = os.path.join(IMC_basedir, 'data/amelia_test')
MODEL_DIR = os.path.join(IMC_basedir, 'data/amelia_model')

In [34]:
# Evaluate the Best Random Search Model
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, Y_test)

Model Performance
Average Error: 0.4713 degrees.
Accuracy = -inf%.


In [35]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of nan%.


/Users/paul.pop/miniconda3/envs/KS3/lib/python3.5/site-packages/ipykernel/__main__.py:1: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


## Grid search

In [ ]:
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a base model
rf = RandomForestRegressor(random_state = 42)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_train, Y_train);

In [ ]:
grid_search.best_params_

In [ ]:
#### Evaluate

In [ ]:

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, test_features, test_labels)

In [ ]:
# Categorical predictions
from keras.utils.np_utils import to_categorical
X = df.copy(deep=True)

Y = np.array(to_categorical(X['life_expectancy_bin']))
# remove columns
X.drop('life_expectancy', axis = 1, inplace=True)
X.drop('life_expectancy_bin', axis = 1, inplace=True)

X = np.array(X)

## Random forest sklearn 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

X, Y = make_regression(n_features=54, n_informative=2,random_state=0, shuffle=False)

regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=2,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [ ]:
print(regr.predict(X_test))

In [1]:
from sknn.mlp import Regressor, Layer
from sklearn.cross_validation import train_test_split

# Build the dataset
sine_X = np.linspace(0, 2 * 2 * math.pi, 1001)  
sine_y = 5 * np.sin(sine_X)
sine_X_train, sine_X_test, sine_y_train, sine_y_test = train_test_split(sine_X, sine_y, test_size=0.33, random_state=42)
# Transform 1D arrays to 2D arrays
sine_X_train = sine_X_train[:,None]
sine_X_test = sine_X_test[:,None]
sine_y_train = sine_y_train[:,None]
sine_y_test = sine_y_test[:,None]

# Build the NN
nn01 = Regressor(layers=[# input layer is added automatically, based on the number of features
                        # a bias unit is added automatically for every input and hiddens layers
                        Layer("Tanh", units=30),      # 1st hidden layer: Rectifier, Sigmoid or Tanh
                        Layer("Tanh", units=30),      # 2nd hidden layer: Rectifier, Sigmoid or Tanh
                        Layer("Linear", units=1)],    # output layer                           
                learning_rate=0.001,
                weight_decay=0.1,                     # weight_decay = regularization
                regularize='L2',
                learning_momentum=0.66,
                n_iter=50,
                batch_size=1,                         # batch_size=1: each sample is treated on its own
                loss_type='mse',
                verbose=True)

# Fitting the model
nn01.fit(sine_X_train, sine_y_train)

# Making the prediction
sine_y_pred = nn01.predict(sine_X_test) 

# Results
print("Results of SKNN Regression:")
print("Residual sum of squares (MSE): %.2f" % np.mean((sine_y_pred - sine_y_test) ** 2))
print('Variance score: %.2f' % nn01.score(sine_X_test, sine_y_test)) # Explained variance score: 1 is perfect prediction

# Plot outputs
plt.clf() 
plt.scatter(sine_X_train, sine_y_train,  color='black', label='training data', alpha=0.1)
plt.scatter(sine_X_test, sine_y_test,  color='red', label='test data')
plt.scatter(sine_X_test, sine_y_pred, color='blue', label='NN prediction') ## NN
plt.grid(True)
plt.legend()
plt.show()

/Users/paul.pop/miniconda3/envs/KS3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


NameError: name 'np' is not defined

#### Randomized Search for hyperparameter tuning

In [ ]:
# Specify parameters and distributions to sample from
param_dist = {  "learning_rate"     : np.logspace(-4, -2, 3),  # 3 numbers from 1e-4 to 1e-2
                "weight_decay"      : np.logspace(-4, -2, 3),
                "learning_momentum" : [0.33, 0.66, 0.99],
                "n_iter"            : [30, 40, 50]}

# Start the Randomized Search
n_iter_search = 20
random_search = RandomizedSearchCV(nn01, param_distributions=param_dist, n_iter=n_iter_search, 
                                   scoring='mean_squared_error', n_jobs=-1, cv=3, verbose=3)

random_search = random_search.fit(sine_X_train, sine_y_train)
print "Best parameters set found on development set:"
print random_search.best_score_, random_search.best_params_
print
print "Grid scores on development set:"
for params, mean_score, scores in random_search.grid_scores_:
    print "%0.3f (+/-%0.03f) for %r" % (mean_score, scores.std() * 2, params)

In [ ]:
nn01_opt = Regressor(layers=[# input layer is added automatically, based on the number of features
                             # a bias unit is added automatically for every input and hiddens layers
                             Layer("Tanh", units=30),      # 1st hidden layer: Rectifier, Sigmoid or Tanh
                             Layer("Tanh", units=30),      # 2nd hidden layer: Rectifier, Sigmoid or Tanh
                             Layer("Linear", units=1)],    # output layer                           
                learning_rate=random_search.best_params_['learning_rate'],  
                weight_decay=random_search.best_params_['weight_decay'],      
                regularize='L2',
                learning_momentum=random_search.best_params_['learning_momentum'],
                n_iter=random_search.best_params_['n_iter'],
                batch_size=1,    # batch_size=1: each sample is treated on its own
                loss_type='mse',
                verbose=True)

# Fitting the model
nn01_opt.fit(sine_X_train, sine_y_train)

# Plot outputs
plt.clf() 
plt.scatter(sine_X_train, sine_y_train,  color='black', label='training data', alpha=0.1)
plt.scatter(sine_X_test, sine_y_test,  color='red', label='test data')
plt.scatter(sine_X_test, nn01_opt.predict(sine_X_test) , color='blue', label='NN prediction') ## NN
plt.grid(True)
plt.legend()
plt.show()
print "Residual sum of squares (MSE): %.2f" % np.mean((nn01_opt.predict(sine_X_test) - sine_y_test) ** 2)

#### Learning curves

In [ ]:
title = 'Learning Curves (NN with tuned hyperparameters)'    
estimator = nn01_opt # regressor with tuned hyperparameters 
plot_learning_curve(estimator, title, sine_X_train, sine_y_train, 
                    ylim=(-10., 0.), cv=5, n_jobs=-1, scoring='mean_squared_error')
plt.grid(True)
plt.show()

Note: It outputs the negative of the MSE, as it always tries to maximize the score. 